In [1414]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math  
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

## **Обработка данных, подготовка выборки**

**1. В основе выборки будет лежать наш изначальный датасет `raw_data`**

In [1418]:
alldata = pd.read_csv('/Users/xexinette/Downloads/raw_data.csv')
alldata.head(1)

,Unnamed: 0,Year,Fest Name,Genre,Film,Director,Nomination,Award,Winner
0,0,2005,카이로 국제영화제,일반(종합),"제니, 주노",김호준,NaN,NaN,NaN


**2. Возьмем созданный на предыдущем этапе датафрейм `yearly_aw`**

Сейчас нас интересуют из него 2 колонки:
- **`Best_Director`** (**BDA**) – режиссер, получивший больше всего наград в этом году.    
- **`Most_Shown_Director`** (**MSD**) – режиссер, чьи фильмы чаще всего участвовали в фестивалях в этом году (независимо от наград).  



In [1421]:
data = pd.read_csv('/Users/xexinette/Downloads/sup_table.csv')
data.head(1)

,Year,Awards,Winners,Win_to_Aw,Rec_num,Fest_Unum,Film_UNnum,Best_Director,BDAll_num,BDA_num,BDALL_AW_Ration,BDA_Share,Most_Shown_Director,MSDW_num,MSDW_Share
0,2000,85,70,0.823529,354,122,111,박찬욱,15,13,0.866667,0.185714,이창동,20,0.18018


### **3. Создадим новые признаки:**

* **Постараемся определить `страну проведения фестивалей` хотя бы для `половины` записей.**

Для этого:

а) Сначала избавимя от дублирующихся фестивалей, которые могли появиться в результате проставленных пробелов (в кор. языке, особенно в названиях, часто допустимо ставить или не ставить пробел на усмотрение пишущего);

In [1424]:
alldata['Fest Name'] = alldata['Fest Name'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)

б) Соберем словарь с название стран для фестивалей по ключевым словам - названиям корейских городов;

In [1427]:
festival_to_country = {}
korean_cities = ['서울', '부산', '대구', '전주', '광주', '인천', '제주', '강릉', '대전', '대한민국']
for x in alldata['Fest Name']:
    for city in korean_cities:
        if city in x:
            festival_to_country[x] = 'South Korea'

в) Смотрим, какие фестивали из `топ-50` по числу записей о них не попали в словарь и **вручную** заносим их туда;

In [1430]:
top_50_fest = alldata['Fest Name'].value_counts().head(50).reset_index()
for i in top_50_fest['Fest Name']:
    if i not in festival_to_country.keys():
        print(i)

부천국제판타스틱영화제
대단한단편영화제
피렌체한국영화제
미쟝센단편영화제
정동진독립영화제
DMZ국제다큐멘터리영화제
파리한국영화제
청룡영화상
제천국제음악영화제
판타지아국제영화제
런던한국영화제
무주산골영화제
여성인권영화제
우디네극동영화제
인디다큐페스티발
백상예술대상
한국영화평론가협회상
뉴욕아시안영화제
인디포럼
광화문국제단편영화제
대종상영화제
부천국제애니메이션페스티벌
헝가리한국영화제
황금촬영상시상식
전북독립영화제
시체스국제판타스틱영화제
한국퀴어영화제
부일영화상
상하이국제영화제


In [1432]:
festival_to_country.update({
    '부천국제판타스틱영화제': 'South Korea',
    '케이프타운세계영화제': 'South Africa',  # некоторые фестивали не были в топ-50, но их легко было заметить при просмотре 
                                        # датасета, так что их тоже вносим для увеличения выборки
    '대단한단편영화제': 'South Korea', 
    '미쟝센단편영화제': 'South Korea',
    '정동진독립영화제': 'South Korea',
    'DMZ국제다큐멘터리영화제': 'South Korea',
    '청룡영화상': 'South Korea',
    '제천국제음악영화제': 'South Korea',
    '네팔국제영화제': 'Nepal',
    '판타지아국제영화제': 'Canada',
    '무주산골영화제': 'South Korea',
    '여성인권영화제': 'South Korea',
    '우디네극동영화제': 'Italy',
    'FairyTales퀴어영화제': 'Canada',
    '인디다큐페스티발': 'South Korea',
    '시애틀영화제': 'USA',
    '백상예술대상': 'South Korea',
    '뉴욕아시안영화제': 'USA',
    '인디포럼': 'South Korea',
    '광화문국제단편영화제': 'South Korea',
    '대종상영화제': 'South Korea',
    '부천국제애니메이션페스티벌': 'South Korea',
    '황금촬영상시상식': 'South Korea',
    '전북독립영화제': 'South Korea',
    '시체스국제판타스틱영화제': 'Spain',
    '부일영화상': 'South Korea',
    '상하이국제영화제': 'China',
    '피렌체한국영화제': 'Italy',
    '파리한국영화제': 'France',
    '런던한국영화제': 'England',
    '한국영화평론가협회상': 'South Korea',
    '헝가리한국영화제': 'Hungary',
    '한국퀴어영화제': 'South Korea'
})

len(festival_to_country) 

109

Загрузим также датасет с обработанными данными только по победителям фестивалей, запишем в датасет `df`

In [1435]:
df = pd.read_csv('/Users/xexinette/Downloads/fixed_data.csv')
df['Fest Name'] = df['Fest Name'].apply(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
df.head(1)

,Unnamed: 0,Year,Fest Name,Genre,Film,Director,Award,Winner
0,17,2005,디렉터스컷어워즈,일반(종합),너는 내 운명,박진표,올해의 여자배우상,전도연


Найдем топ-3 режиссера и для них определим страны фестивалей, в которых они побеждали

In [1438]:
top_3 = list(df['Director'].value_counts().head(3).index)
top_3

['봉준호', '박찬욱', '이준익']

In [1440]:
df_top_3_dir = df[df['Director'].isin(top_3)]
df_top_3_dir['Fest_Country'] = df_top_3_dir['Fest Name'].map(festival_to_country)
missing_country = df_top_3_dir[df_top_3_dir['Fest_Country'].isna()]
missing_country['Fest Name'].unique()

array(['디렉터스컷어워즈', '홍콩아시안영화제', '춘사영화제', '칸국제영화제', '맥스무비최고의영화상', '여성영화인축제',
       '산세바스티안국제영화제', '한국영화제작가협회상', 'LA영화비평가협회상', '인도국제영화제', '아시안필름어워드',
       '마리끌레르영화제', '영국아카데미시상식', '마카오국제영화제', '아시아퍼시픽스크린어워즈',
       '오슬로필름즈프롬더사우스영화제', '제네바국제영화제', '상파울루국제영화제', '캘거리국제영화제', '판타스틱페스트',
       '토론토국제영화제', '로카르노국제영화제', '유라시아국제영화제', '시드니영화제', '아카데미시상식',
       '로테르담국제영화제', '미국배우조합상', '트롬쇠국제영화제', '골든글로브어워즈', '디스커싱필름비평가협회상'],
      dtype=object)

In [1442]:
festival_to_country.update({
    '디렉터스컷어워즈': 'South Korea',
    '홍콩아시안영화제': 'Hong Kong',
    '춘사영화제': 'South Korea',
    '칸국제영화제': 'France',
    '맥스무비최고의영화상': 'South Korea',
    '여성영화인축제': 'South Korea',
    '산세바스티안국제영화제': 'Spain',
    '한국영화제작가협회상': 'South Korea',
    'LA영화비평가협회상': 'USA',
    '인도국제영화제': 'India',
    '아시안필름어워드': 'Hong Kong',
    '마리끌레르영화제': 'South Korea',
    '영국아카데미시상식': 'England',
    '마카오국제영화제': 'China',
    '아시아퍼시픽스크린어워즈': 'Australia',
    '오슬로필름즈프롬더사우스영화제': 'Norway',
    '제네바국제영화제': 'Switzerland',
    '상파울루국제영화제': 'Brazil',
    '캘거리국제영화제': 'Canada',
    '판타스틱페스트': 'USA',
    '토론토국제영화제': 'Canada',
    '로카르노국제영화제': 'Switzerland',
    '유라시아국제영화제': 'Online',
    '시드니영화제': 'Australia',
    '아카데미시상식': 'USA',
    '로테르담국제영화제': 'Netherlands',
    '미국배우조합상': 'USA',
    '트롬쇠국제영화제': 'Norway',
    '골든글로브어워즈': 'USA',
    '디스커싱필름비평가협회상': 'Online'
})
    
len(festival_to_country)               

139

In [1444]:
alldata['Fest_Country'] = alldata['Fest Name'].map(festival_to_country)

**Итак, у нас получилось определить страну для 139 фестивалей (в том числе для топ-50 по популярности и все, где побеждали топ-3 режиссера)**


г) Сопоставляем полученые страны с данными в датасете, проверяем на пропуски через .info()

In [1447]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37665 entries, 0 to 37664
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    37665 non-null  int64 
 1   Year          37665 non-null  int64 
 2   Fest Name     37665 non-null  object
 3   Genre         37591 non-null  object
 4   Film          37520 non-null  object
 5   Director      35261 non-null  object
 6   Nomination    31872 non-null  object
 7   Award         6250 non-null   object
 8   Winner        5213 non-null   object
 9   Fest_Country  23100 non-null  object
dtypes: int64(2), object(8)
memory usage: 2.9+ MB


видим:
> *9 Festival_Country 23100 non-null object,*

*значит мы смогли определить страну проведения для `61%` (23100/37665) записей*

д) Добавим бинарный признак `Is_National` - 'Национальный / Зарубежный' фестиваль (1/0)

In [1451]:
alldata['Is_National'] = alldata['Fest_Country'].apply(lambda x: 1 if x == 'South Korea' else 0).astype(int)

* **Теперь создадим систему рангов, в соответствии с которой разделим режиссеров:**

> 1 ранг: режиссер хотя бы раз попадал в список лучших `Best_Director`;

> 2 ранг: режиссер хотя бы раз попадал в список самых часто показываемых на фестивалях `Most_Shown_Director`;

> 3 ранг: режиссер ни разу не был ни в одном из списков.

In [1454]:
best_directors = set(data['Best_Director'].dropna())
most_shown_directors = set(data['Most_Shown_Director'].dropna())

director_rank = {}
for director in most_shown_directors:
    director_rank[director] = 2
for director in best_directors:
    director_rank[director] = 1

alldata["director_rank"] = alldata["Director"].map(director_rank).fillna(3).astype(int) 
# добавим новый признак к нашему датасету

Для каждого фильма рассчитаем бинарную переменную `awarded`:
* `1` — фильм получил награду (на основе непустого поля `Winner`);
* `0` — фильм не получил награду.


In [1457]:
alldata['awarded'] = alldata['Winner'].notna().astype(int)

Избавимся от строк с пропущенными значениями стран и режиссеров

In [1460]:
alldata = alldata.dropna(subset=['Fest_Country', 'Director'])

**Теперь создадим дополнительную таблицу, в которой соберем данные по каждому фильму, а именно:**

- в `Total_Presentations` внесем общее кол-во записей об этом фильме в нашем датасете;
- в `Unique_Festivals` кол-во уникальных фестивалей, на которых фильм был представлен за все время;
- в `Unique_Years` кол-во лет, когда фильм участвовал в фестах;
- в `Director` режиссер;
- в `National_Count` сколько из фестивалей, в которых участвовал фильм, проводились в Корее;
- в `International_Count` сколько из фестивалей, в которых участвовал фильм, проводились зарубежом;
- в `Award_Count` кол-во завоеванных наград.

In [1463]:
film_counts = alldata['Film'].value_counts().reset_index()
film_counts.columns = ['Film', 'Total_Presentations']

unique_fests = alldata.groupby('Film')['Fest Name'].nunique().reset_index()
unique_fests.columns = ['Film', 'Unique_Festivals']

unique_years = alldata.groupby('Film')['Year'].nunique().reset_index()
unique_years.columns = ['Film', 'Unique_Years']

film_directors = alldata[['Film', 'Director']].drop_duplicates(subset='Film')

national_counts = alldata[alldata['Is_National'] == 1].groupby('Film').size().rename('National_Count')
international_counts = alldata[alldata['Is_National'] == 0].groupby('Film').size().rename('International_Count')
festival_origin_summary = pd.concat([national_counts, international_counts], axis=1).fillna(0).astype(int)

awards_counts = alldata[alldata['Winner'].notna()]['Film'].value_counts().reset_index()
awards_counts.columns = ['Film', 'Award_Count']

film_summary = film_counts.merge(unique_fests, on='Film').merge(unique_years, on='Film').merge(film_directors, on='Film', how='left').merge(festival_origin_summary, on='Film', how='left').merge(awards_counts, on='Film', how='left')

Совершим необходимые преобразования:
> заменим пустые значения в кол-ве наград на 0;
> посчтитаем соотношение наград к общему кол-ву записей;
> добавим в таблицу ранг режиссера.

- в `Awards_ratio` соотношение наград к участию;
- в `Director_Rank` ранг режиссера.

In [1466]:
film_summary['Award_Count'] = film_summary['Award_Count'].fillna(0).astype(int)
film_summary['Awards_ratio'] = film_summary['Award_Count'] / film_summary['Total_Presentations']
film_summary = film_summary[film_summary['Total_Presentations'] > 5]
film_summary['Director_Rank'] = film_summary['Director'].map(director_rank).fillna(3).astype(int)
film_summary.head(1)

,Film,Total_Presentations,Unique_Festivals,Unique_Years,Director,National_Count,International_Count,Award_Count,Awards_ratio,Director_Rank
0,기생충,88,36,4,봉준호,38,50,76,0.863636,1


In [1468]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21446 entries, 2 to 37664
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     21446 non-null  int64 
 1   Year           21446 non-null  int64 
 2   Fest Name      21446 non-null  object
 3   Genre          21446 non-null  object
 4   Film           21446 non-null  object
 5   Director       21446 non-null  object
 6   Nomination     17836 non-null  object
 7   Award          4330 non-null   object
 8   Winner         3962 non-null   object
 9   Fest_Country   21446 non-null  object
 10  Is_National    21446 non-null  int64 
 11  director_rank  21446 non-null  int64 
 12  awarded        21446 non-null  int64 
dtypes: int64(5), object(8)
memory usage: 2.3+ MB


**НАША ВЫБОРКА ГОТОВА :)**

Теперь можем переходить к выдвижению и проверке гипотез

---

## **Формулировка и проверка гипотез**

## Гипотеза №1: влияние ранга режиссера на вероятность получения награды

Мы хотим проверить, влияет ли "ранг" режиссера на вероятность получения им награды на фестивале. Будем использовать данные по фильмам режиссеров 1-го и 3-го ранга из датафрейма `alldata`, генеральная совокупность: все корейские фильмы, которые были или будут выпущены.

* дисперсия по генеральной совокупности неизвестна, поэтому будем использовать **t-статистику**;
* **мотивация**: проверка данной гипотезы позволит судить, насколько "именитость" режиссера влияет на успех картины на фестивале.


1. **Формулируем гипотезы**
<div align="center">

$$
H_0: \mu_1 = \mu_3
$$

$$
H_1: \mu_1 \ne \mu_3
$$

</div>


- **H₀ (нулевая гипотеза):**  
  Средняя вероятность получения награды у фильмов от режиссёров с рангом 1 **равна** таковой у фильмов с рангом 3.

- **H₁ (альтернативная гипотеза):**  
  Средняя вероятность **не равна** — есть статистически значимая разница между рангами.

---
                      



2. **Выбираем уровень значимости**
<div align="center">

$$
\alpha = 0.01
$$

</div>



3. **Считаем p-value**

In [1475]:
from scipy.stats import ttest_ind

rank1 = alldata[alldata['director_rank'] == 1]['awarded']
rank3 = alldata[alldata['director_rank'] == 3]['awarded']

result = ttest_ind(rank1, rank3, alternative="two-sided", equal_var=False)
result

TtestResult(statistic=31.831044796900773, pvalue=9.623770511682689e-170, df=1476.809224568856)

4. **Результаты**

- t-статистика: 31.83  

- Уровень значимости:  
  <div align="center">

  $$
  \alpha = 0.01
  $$

  </div>

- **p-value:**  
  <div align="center">

  $$
  p = 9.62 \times 10^{-170}
  $$

  </div>

---

### Вывод:

Так как:

<div align="center">

$$
p \ll \alpha
$$

</div>

мы отвергаем нулевую гипотезу и заключаем, что вероятность получения награды **значимо отличается** у фильмов режиссёров ранга 1 и ранга 3.



### Найдем средние и доверительные интервалы вероятностей режиссеров 1-го и 3-го ранга получить награду на фестивале:

In [1479]:
mean_rank1 = rank1.mean()
sigma1 = rank1.std(ddof=1)
n_rank1 = len(rank1)
conf_interval = stats.t.interval(0.99, df=len(rank1)-1, loc=mean_rank1, scale=sigma1/np.sqrt(len(rank1)))

mean_rank1, conf_interval

(0.5874635568513119, (0.5531690170488719, 0.6217580966537519))

Мы с 99% уверенностью можем утверждать, что средняя вероятность получения награды для фильмов от режиссёров **с рангом 1** лежит в диапазоне от 55.3% до 62.1%.

In [1482]:
mean_rank3 = rank3.mean()
sigma3 = rank3.std(ddof=1)
n_rank3 = len(rank3)
conf_interval = stats.t.interval(0.99, df=len(rank3)-1, loc=mean_rank3, scale=sigma3/np.sqrt(len(rank3)))

mean_rank1, conf_interval

(0.5874635568513119, (0.14963576580501625, 0.16297542720062602))

Мы с 99% уверенностью можем утверждать, что средняя вероятность получения награды для фильмов от режиссёров **с рангом 3** лежит в диапазоне от 15% до 16.3%.

**ИТОГ:** Судя по средним, вероятность получить награду на фестивале значительно выше у режиссеров 1-го ранга.

---

## Гипотеза №2: 
**На успешность фильма (award_ration) значительно влияет кол-во лет, на протяжении которых фильм участвует в фестивалях; кол-во уникальных фестивалей, на которых он показывается; а также место проведения фестиваля (Корея / зарубеж).**

Мы хотим проверить, какие еще факторы влияют на вероятность получить награду на фестивале. Для этого построим многофакторную регрессию.

## Многофакторная линейная Регрессия

В рамках исследования мы стремимся понять:

- Что именно влияет на успешность фильма на фестивалях?
- Достаточно ли просто часто участвовать как можно чаще, или важна уникальность и длительность присутствия на сцене?
- Какую роль играет место проведения фестивалей — в частности, есть ли различие между фестивалями, проходящими в Южной Корее и за рубежом?

Для этого будем строить многофакторную линейную регрессию по данным из датасета `film_summary`, где:
  > **целевая переменная:** `Awards_ratio` - соотношение наград к участию;;

  > **независимые переменные:**
        > `Unique_Festivals` - кол-во уникальных фестивалей, на которых фильм был представлен за все время;
        > `National_Count` -  сколько из фестивалей, в которых участвовал фильм, проводились в Корее; `International_Count` - сколько из фестивалей, в которых участвовал фильм, проводились зарубежом; `Unique_Years` - кол-во лет, когда фильм участвовал в фестах;

---


**Нулевая гипотеза (H₀):** Все коэффициенты при признаках равны нулю — признаки не влияют на Awards_ratio.

<div align="center">

$$
H_0: \beta_1 = \beta_2 = \beta_3 = \beta_4 = 0 \\
$$

</div>

**Альтернативная гипотеза (H₁):** Хотя бы один из коэффициентов регрессии значимо отличается от нуля, то есть хотя бы один признак влияет на Awards_ratio.

<div align="center">

$$
H_1: \text{Хотя бы один из } \beta_i \ne 0
$$

</div>



**Сначала обучим модель, для обучения возьмем 70% данных**

In [1492]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import statsmodels.api as sm

X = film_summary[['Unique_Festivals', 'National_Count', 'International_Count', 'Unique_Years']] 
y = film_summary['Awards_ratio'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()
model.params



const                  0.276293
Unique_Festivals      -0.033207
National_Count         0.038029
International_Count    0.031751
Unique_Years          -0.030722
dtype: float64

**А теперь протестируем модель и рассчитаем метрики:**

In [1495]:
y_pred = model.predict(X_test_const)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("MAE:", mae)
print("R²:", r2)

MSE: 0.04318591852168316
MAE: 0.17412258950307527
R²: 0.3041760735590944


**Воспользуемся методом .summary(), чтобы посмотреть F-статистику и результат теста Уальда**

In [1498]:
hypothesis = 'Unique_Festivals = 0, National_Count = 0, International_Count = 0, Unique_Years = 0'
wald_result = model.wald_test(hypothesis)
print(wald_result)


<F test: F=array([[49.77479866]]), p=1.0187669354568515e-35, df_denom=546, df_num=4>


## Результаты
* **F-statistic:**	49.77
* **p** = 1.02e-35


Т.к. p-value меньше заданного уровня значимости 0.01, мы отвергаем нулевую гипотезу о том, что все 4 независимых фактора незначимы одновременно и принимаем, что хотя бы один из них оказывает статистически значимое влияние на вероятность успеха фильма.

**Обратимся к полученным коэффициентам:**
> **const:**                  0.276293;

если все предикторы = 0, шанс фильма на успех = `27.6%`

> **Unique_Festivals:**      -0.033207

чем больше фестивалей "посещает" фильм, тем ниже его доля успеха; каждый доп. фест. снижает award_ratio на `3.3%`

> **National_Count:**         0.038029

"Домашние" фестивали более радушны к корейским картинам, участие в корейском фестивале повышает award_ratio на `3.8%`, что незначительно, но все же выше

> **International_Count:**    0.031751

`3.2%`, получаемых на зарубежных фестивалях.

> **Unique_Years:**          -0.030722

Чем дольше фильм продолжает участвовать в фестивалях, тем нижк его доля успеха, т.к. дополнительный год участия связан со снижением award_ratio на `3.1%`

**ИТОГ: да, коэффициенты небольшие, но статистически значимые.**

Благодаря полученным инсайтам мы может сделать некоторые выводы об опыте презентации корейских кинокартин на домащней и зарубежной арене:



### Выводы

Согласно результатам регрессии и F-тесту Уальда (F = 49.77, p < 0.0001), модель в целом является значимой, и хотя бы один из факторов влияет на результат. Таким образом, гипотеза №2 подтверждается.

* Все четыре переменные оказались статистически значимыми (p-value < 0.01)

Для улучшения модели, можно поиграться с кол-вом факторов, добавить также категориальный признак (ранг режиссера) и попробовать построить логистическую регрессию, или попробовать изменить масштаб выборки.

***Но у меня, к сожалению, не хватило времени на это, очень много было потрачено на обработку данных и создание новых признаков ㅠㅠ***